# **Pre procesamiento de la base de datos**: creación de las variables de interés

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder


In [2]:
educ = pd.read_csv("../data/raw/Enaho01a-2021-300.csv", encoding="iso-8859-1")


C:\Users\User\AppData\Local\Temp\ipykernel_30724\1480248929.py:1: DtypeWarning: Columns (12,13,26,32,59,346,510) have mixed types. Specify dtype option on import or set low_memory=False.
  educ = pd.read_csv("../data/raw/Enaho01a-2021-300.csv", encoding="iso-8859-1")


In [3]:
r= educ[['P304A','P308A'] ]

**Creacion de la variable dummy dependiente: Deserción Escolar**

In [4]:


# * DESERCION ESCOLAR
#  nos quedamos con los de la Educación Básica Regular
educ = educ[(educ['P304A'] >= '1') & (educ['P304A'] <= '3')]
#P304B representa el año en el que estaba matriculado la persona el año pasado (2020). 1,2,3,4,5 son los años de secundaria. 1,2,3 son los años de educación inicial. 0 son los de Primaria porque ellos están representados en "grados" y no en años
educ = educ[(educ['P304B'] >= '0') & (educ['P304B'] <= '5')]
#se elimina a los que estuvieron el año pasado en quinto de secundaria y que aprobaron el año
educ = educ.drop(educ[(educ['P304B'] == 5) & (educ['P305'] == 1)].index)
#se elimina a los que tienen 4 años en la actualidad (2021) porque no se tenia registro si estuvieron matriculados el año pasado
educ = educ[educ['P208A'] != 4]
educ['P308A'] = pd.to_numeric(educ['P308A'], errors='coerce')
#creacion de Deserción escolar para las personas que estuvieron matriculados el año pasado. 1= para las personas que estuvieron matriculados el año pasado y que actualmente ya no lo están. 0= caso contrario
educ['DESERCION_ESCOLAR'] = (pd.notna(educ['P304A']) & pd.isna(educ['P308A'])).astype('int')




In [5]:

# * sexo (1= Hombre, 0 = Mujer)
educ['SEXO'] = 0  
educ.loc[educ['P207'] == 1, 'SEXO'] = 1


# * Edad
educ.rename(columns={'P208A': 'EDAD'}, inplace=True)


# * Area Geográfica (1=Urbano, 0= Rural)
educ['AREA_GEOGRAFICA'] = 1
educ.loc[(educ['ESTRATO'] >= 6) & (educ['ESTRATO'] <= 8), 'AREA_GEOGRAFICA'] = 0


#* Lengua Materna (1=Castellano, 0=Lengua Nativa)
educ['LENGUA_MATERNA'] = np.nan  
educ.loc[educ['P300A'] == 4, 'LENGUA_MATERNA'] = 1
educ.loc[educ['P300A'].isin([1, 2, 3, 10, 11, 12, 13, 14, 15]), 'LENGUA_MATERNA'] = 0
educ = educ.dropna(subset=['LENGUA_MATERNA'])


#*Tipo de Colegio (1=Colegio Público, 0=Colegio Privado)
label_encoder = LabelEncoder()
educ['P308D'] = label_encoder.fit_transform(educ['P308D'])
educ['TIPO_DE_COLEGIO'] = 0   
educ.loc[educ['P308D'] != 2 , 'TIPO_DE_COLEGIO'] = 1

#* Programas sociales (1=si recibio recursos de algún programa social, 0=caso contrario)
label_encoder = LabelEncoder()
educ['P311A6$4'] = label_encoder.fit_transform(educ['P311A6$4'])
educ['P311A6$8'] = label_encoder.fit_transform(educ['P311A6$8'])
educ['P311A6$9'] = label_encoder.fit_transform(educ['P311A6$9'])

# Creamos una variable dummy 'programa_utiles' 
educ['programa_utiles'] = 0  
educ.loc[educ['P311A6$4'] == 2, 'programa_utiles'] = 1

educ['programa_utiles'].value_counts()
# Crear la columna 'programa' basada en las condiciones dadas
educ['PROGRAMA'] = (educ['P311A6$8'] | educ['P311A6$9']| educ['programa_utiles']).astype(int)



variables = ['AÑO', 'CONGLOME', 'VIVIENDA', 'HOGAR', 'CODPERSO', 'UBIGEO','DESERCION_ESCOLAR', 
             'SEXO', 'EDAD', 'AREA_GEOGRAFICA', 'LENGUA_MATERNA', 'TIPO_DE_COLEGIO', 'PROGRAMA'
               ]
data = educ[variables]

In [6]:
sumaria = pd.read_csv("../data/raw/Sumaria-2021.csv", encoding="iso-8859-1")
hogar = pd.read_csv("../data/raw/Enaho01-2021-100.csv", encoding="iso-8859-1")
gob = pd.read_csv("../data/raw/Enaho01B-2021-2.csv", encoding="iso-8859-1")
empleo = pd.read_csv("../data/raw/Enaho01a-2021-500.csv", encoding="iso-8859-1")
salud = pd.read_csv("../data/raw/Enaho01a-2021-400.csv", encoding="iso-8859-1")


C:\Users\User\AppData\Local\Temp\ipykernel_30724\1594343561.py:2: DtypeWarning: Columns (323) have mixed types. Specify dtype option on import or set low_memory=False.
  hogar = pd.read_csv("../data/raw/Enaho01-2021-100.csv", encoding="iso-8859-1")
C:\Users\User\AppData\Local\Temp\ipykernel_30724\1594343561.py:3: DtypeWarning: Columns (10,50,51,52,96) have mixed types. Specify dtype option on import or set low_memory=False.
  gob = pd.read_csv("../data/raw/Enaho01B-2021-2.csv", encoding="iso-8859-1")
C:\Users\User\AppData\Local\Temp\ipykernel_30724\1594343561.py:4: DtypeWarning: Columns (229,234,239,247,252,257,262,267,272,277,282,287,292,297,302,307,312,317,322,327,332,337,342,347,354,358,362,366,370,431,434,437,440,443,446,449,452,457,459,461,463,465,467,469,473,474,475,476,477,481,573,574,575,576,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1103,1106,1111,1456) have mixed types. Specify dtype option on import or set low_memory=False.
  empleo = pd.read_csv("../data/raw/Enaho01a

In [7]:

#* SITUACION LABORAL (1= si trabaja , 0 = caso contrario)
empleo = empleo[empleo['OCU500'] != 0]
empleo['SITUACION_LABORAL'] = 0  
empleo.loc[empleo['OCU500'] == 1, 'SITUACION_LABORAL'] = 1

variables = ['AÑO', 'CONGLOME', 'VIVIENDA', 'HOGAR', 'CODPERSO', 'UBIGEO', 'SITUACION_LABORAL']
empleo = empleo[variables]

data = data.merge(empleo, how='left', on=['AÑO', 'CONGLOME', 'VIVIENDA',  'HOGAR', 'CODPERSO', 'UBIGEO'])
data['SITUACION_LABORAL'] = data['SITUACION_LABORAL'].fillna(0)

In [8]:
# Codificamos la variable
salud['P414$17'] = salud['P414$17'].astype(str)
label_encoder = LabelEncoder()
salud['P414$17'] = label_encoder.fit_transform(salud['P414$17'])

# Quitamos dos categorias de la variable
salud['P414$17'] = salud['P414$17'].astype(str)
categorias_a_eliminar = ['0', '3']
salud = salud[~salud['P414$17'].isin(categorias_a_eliminar)]

In [9]:

#*  variable dummy de "vacunas_Covid" (1= si se ha vacunado, 0 = caso contrario)
cambios_etiquetas = {'2': 0, '1': 1}
salud['VACUNAS_COVID'] = salud['P414$17'].replace(cambios_etiquetas)

salud['VACUNAS_COVID'].value_counts()

VACUNAS_COVID
0    90708
1    19845
Name: count, dtype: int64

In [10]:
variables = ['AÑO', 'CONGLOME', 'VIVIENDA', 'HOGAR','CODPERSO', 'UBIGEO', 'VACUNAS_COVID']
salud = salud[variables]
data = data.merge(salud, how='left', on=['AÑO', 'CONGLOME', 'VIVIENDA',  'HOGAR','CODPERSO', 'UBIGEO'])

In [11]:

#* Educacion del padre (1= sin educacion, 2= Primaria, 3= Secundaria, 4= Superior)
label_encoder = LabelEncoder()
gob['P45_1'] = label_encoder.fit_transform(gob['P45_1'])


# Supongamos que educ es tu DataFrame
gob['EDUCACION_PADRE'] = np.nan

gob.loc[gob['P45_1'] == 1, 'EDUCACION_PADRE'] = 1
gob.loc[gob['P45_1'].isin([2, 3]), 'EDUCACION_PADRE'] = 2
gob.loc[gob['P45_1'].isin([4, 5]), 'EDUCACION_PADRE'] = 3
gob.loc[gob['P45_1'].isin([6, 7, 8, 9]), 'EDUCACION_PADRE'] = 4


#* Educacion de la madre (1= sin educacion, 2= Primaria, 3= Secundaria, 4= Superior)
label_encoder = LabelEncoder()
gob['P45_2'] = label_encoder.fit_transform(gob['P45_2'])


# Supongamos que educ es tu DataFrame
gob['EDUCACION_MADRE'] = np.nan

# Asignar valores según las condiciones especificadas
gob.loc[gob['P45_2'] == 1, 'EDUCACION_MADRE'] = 1
gob.loc[gob['P45_2'].isin([2, 3]), 'EDUCACION_MADRE'] = 2
gob.loc[gob['P45_2'].isin([4, 5]), 'EDUCACION_MADRE'] = 3
gob.loc[gob['P45_2'].isin([6, 7, 8, 9]), 'EDUCACION_MADRE'] = 4



variables = ['AÑO', 'CONGLOME', 'VIVIENDA', 'HOGAR','UBIGEO' ,'EDUCACION_PADRE', 'EDUCACION_MADRE']
gob = gob[variables]

data = data.merge(gob, how='left', on=['AÑO', 'CONGLOME', 'VIVIENDA', 'HOGAR', 'UBIGEO'])



In [12]:

#* TAMAÑO FAMILIAR
sumaria.rename(columns={'MIEPERHO': 'TAMAÑO_FAMILIAR'}, inplace=True)


#* INGRESO FAMILIAR
sumaria['INGRESO_FAMILIAR'] = sumaria['INGHOG2D'] / (sumaria['TAMAÑO_FAMILIAR'] * 12)

variables = ['AÑO', 'CONGLOME', 'VIVIENDA', 'HOGAR', 'UBIGEO','INGRESO_FAMILIAR','TAMAÑO_FAMILIAR' ]
sumaria = sumaria[variables]

data = data.merge(sumaria, how='left', on=['AÑO', 'CONGLOME', 'VIVIENDA', 'HOGAR'])




In [13]:

#*RECURSOS TECNOLOGICOS

# *CELULAR
hogar['P1142'] = hogar['P1142'].astype('str')
hogar['P1142'] = hogar['P1142'].replace({' ': '999'})
hogar['P1142'] = hogar['P1142'].astype('int')
hogar['P1142'] = hogar['P1142'].replace({999: np.nan})
hogar['P1142'] = hogar['P1142'].fillna(0)
hogar['CELULAR'] = hogar['P1142']

# *TV
hogar['P1143'] = hogar['P1143'].astype('str')
hogar['P1143'] = hogar['P1143'].replace({' ': '999'})
hogar['P1143'] = hogar['P1143'].astype('int')
hogar['P1143'] = hogar['P1143'].replace({999: np.nan})
hogar['P1143'] = hogar['P1143'].fillna(0)
hogar['TV'] = hogar['P1143']


# * INTERNET
hogar['P1144'] = hogar['P1144'].astype('str')
hogar['P1144'] = hogar['P1144'].replace({' ': '999'})
hogar['P1144'] = hogar['P1144'].astype('int')
hogar['P1144'] = hogar['P1144'].replace({999: np.nan})
hogar['P1144'] = hogar['P1144'].fillna(0)
hogar['INTERNET'] = hogar['P1144']


variables = ['AÑO', 'CONGLOME', 'VIVIENDA', 'HOGAR', 'INTERNET', 'TV', 'CELULAR' ]
hogar = hogar[variables]

data = data.merge(hogar,  how='left', on=['AÑO', 'CONGLOME', 'VIVIENDA', 'HOGAR'])

In [14]:
data=data.dropna()

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18514 entries, 0 to 19371
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   AÑO                18514 non-null  int64  
 1   CONGLOME           18514 non-null  int64  
 2   VIVIENDA           18514 non-null  int64  
 3   HOGAR              18514 non-null  int64  
 4   CODPERSO           18514 non-null  int64  
 5   UBIGEO_x           18514 non-null  int64  
 6   DESERCION_ESCOLAR  18514 non-null  int32  
 7   SEXO               18514 non-null  int64  
 8   EDAD               18514 non-null  int64  
 9   AREA_GEOGRAFICA    18514 non-null  int64  
 10  LENGUA_MATERNA     18514 non-null  float64
 11  TIPO_DE_COLEGIO    18514 non-null  int64  
 12  PROGRAMA           18514 non-null  int32  
 13  SITUACION_LABORAL  18514 non-null  float64
 14  VACUNAS_COVID      18514 non-null  float64
 15  EDUCACION_PADRE    18514 non-null  float64
 16  EDUCACION_MADRE    18514 no

In [16]:
data['LENGUA_MATERNA'] = data['LENGUA_MATERNA'].astype(int)
data['SITUACION_LABORAL'] = data['SITUACION_LABORAL'].astype(int)
data['EDUCACION_PADRE'] = data['EDUCACION_PADRE'].astype(int)
data['EDUCACION_MADRE'] = data['EDUCACION_MADRE'].astype(int)
data['INTERNET'] = data['INTERNET'].astype(int)
data['TV'] = data['TV'].astype(int)
data['CELULAR'] = data['CELULAR'].astype(int)
data['VACUNAS_COVID'] = data['VACUNAS_COVID'].astype(int)


In [17]:
data = data.rename(columns=str.lower)


In [18]:
data.head()

,año,conglome,vivienda,hogar,codperso,ubigeo_x,desercion_escolar,sexo,edad,area_geografica,...,situacion_laboral,vacunas_covid,educacion_padre,educacion_madre,ubigeo_y,ingreso_familiar,tamaño_familiar,internet,tv,celular
0,2021,5009,98,11,3,10101,1,0,17,1,...,1,0,4,3,10101,1013.349609,4,1,1,1
1,2021,5009,98,11,4,10101,1,1,7,1,...,0,0,4,3,10101,1013.349609,4,1,1,1
2,2021,5019,85,11,3,10101,0,0,8,1,...,0,0,2,2,10101,670.280816,3,0,1,1
3,2021,5019,114,11,4,10101,0,1,9,1,...,0,0,3,3,10101,1056.816569,4,1,1,1
4,2021,5020,102,11,4,10101,0,0,11,1,...,0,0,1,1,10101,1556.165148,6,1,1,1


In [19]:
data.to_csv("../data/final/DATA1.csv")